# answer question originated from unifying process

In [71]:
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from pandas.io.parsers import read_csv
from gis_functions import *
from functions import *



# check original PE
## compare original data with half-way

In [ ]:
# why has karn so much Pe in plus?
import functions
d = r'C:/Users/fabrizio/Documents/R_data/PROJECTS/MASTER/DATA/KKA Kärnten NEU.xlsx'

data, trial = functions.reader(d, 'Data K', header=15, test=False)
data['EW 60']=data['EW 60'].replace('?', 0)
data['EW 60']=data['EW 60'].astype(str)
data.loc[data['EW 60'].str.contains('Bew'), 'EW 60']='0'
data.loc[data['EW 60'].str.contains('Bescheid'), 'EW 60']='0'
data.loc[data['EW 60'].str.contains('m³'), 'EW 60']='0'
data.loc[data['EW 60'].str.contains('Liter'), 'EW 60']='0'
data.loc[data['EW 60'].str.contains('nan'), 'EW 60']='0'

#perfect match. carintia had from beginning so many PEs
data['EW 60'].astype(float).sum()

In [ ]:
sub=data[['Bautyp','EW 60']]
sub['EW 60']=sub['EW 60'].astype(float).round().astype(int)
sub[sub['EW 60']<50].sum()

In [ ]:
halfway=pd.read_excel('half-way/SBG.xlsx')


In [ ]:
halfway
halfway[halfway.PE<50].PE.sum()
#halfway[halfway.EW60<50].EW60.sum()

In [ ]:
#check original PE for LowerA
d = r'C:/Users/fabrizio/Documents/R_data/PROJECTS/MASTER/DATA/KKA NÖ NEU DE final.xlsx'
data, trial = functions.reader(d, 'Data NÖ (Technol)', header=5, test=False)

#again, perfect match
data['EW60_BEWILLIGT'].astype(float).sum()

# check the unmerged 
## why some rows have not been merged?

In [ ]:
# check the not merged of KArn
data=pd.read_excel('final/not_merged_karn.xlsx')

In [ ]:
#zoom on merge key
data.loc[:,['KG', 'KG_NR']]
#one KG , and the rest is 0 that used to be NAs
#OK, differenced between half-way and final are explained

In [ ]:
#all 0 except one
data=pd.read_excel('final/not_merged_sbg.xlsx')
data.loc[:,['KG', 'KG_NR']]


# unify OEBO

In [ ]:
oebo=geopandas.read_file('final/oebo.geojson')


In [ ]:
oebo['no_nitri']=oebo.freq-oebo.NITRIFIZIERUNG

# what the fuck happened with mean_age steiermark?

In [ ]:
steyr=pd.read_excel('half-way/steyr.xlsx')

In [ ]:
steyr.sort_values('year', ascending=True)## 2077
#steyr[steyr.sort_values('year', ascending=True).year>0]## 


In [ ]:
data=geopandas.read_file('final/steyr.geojson')


In [ ]:
data[data.sort_values('mean_year', ascending=True).mean_year>0]## 


# alle the 0s disturb the mean value!! where do 0 come from ? NAs

# inserting new function : sumPE_nonitri

In [ ]:
data=geopandas.read_file('final/noe.geojson')



In [ ]:
data.columns

In [ ]:
data

In [ ]:

data[['PE_nonitri', 'freq','no_nitri', 'sum_PE']].sort_values('PE_nonitri',ascending=False)

In [ ]:

data[['PE_nonitri_y', 'freq','no_nitri', 'sum_PE']].sort_values('PE_nonitri_y',ascending=False)

# check why so many NAs in LOWER austria tech_type

In [ ]:
#check original PE for LowerA
d = r'C:/Users/fabrizio/Documents/R_data/PROJECTS/MASTER/DATA/KKA NÖ NEU DE final.xlsx'
data, trial = functions.reader(d, 'Data NÖ (Technol)', header=5, test=False)

data = cleaner(data, ["Unnamed: 0", "Unnamed: 28",
               "Unnamed: 29", "Unnamed: 30"])

#work on bautyp
bautyp = data[["Mechan.", "Biolog.", "Chemisch", "Unbek."]]
conditions_bau = [
    data["Unbek."] == 1,
    data["Mechan."] == 1,
    data["Biolog."] == 1,
    data["Chemisch"] == 1,
    (data["Biolog."] == 1) & (data["Mechan."] == 1)
]

outcomes_bau = [
    "Unbekannt", "Mech", "Bio", "Chem", "Bio"
]

data = logical_column(conditions_bau, outcomes_bau, data, 'bautyp')
data=col_dropper(data, bautyp)
data

#work on tech_type
col_list = ['3-k', 'Bel.', 'SBR', 'MBR', 'Tropf', 'RBC', 'Fest', 'Wirbel', 'BKF', 'PKA', 'Filtersack',
         'Kompost', 'Andere', 'Unbekannt']



change_colname_index(9, data,col_list)

In [ ]:
data=data.reset_index()

In [ ]:
#same as from excel
len(data[data['Unbekannt']==1])

In [ ]:
#same as later
data[data['Unbekannt']==1]['EW60_BEWILLIGT'].sum()

# high nmber from beginning

In [74]:
#check original PE for LowerA
d = r'C:/Users/fabrizio/Documents/R_data/PROJECTS/MASTER/DATA/KKA NÖ NEU DE final.xlsx'
data, trial = functions.reader(d, 'Data NÖ (KKA) (Technol)', header=5, test=False)
data


,Unnamed: 0,BEZIRK,NAME,EW60_BEWILLIGT,BEWILLIGUNGSJAHR,RECHTSWERT,HOCHWERT,Mechan.,Biolog.,Chemisch,...,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53
0,1,Lilienfeld,ARA Wohnhaus Eberl KG Kerschenbach 1572 LF,4,1983,700050.0,323594.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,,,
1,1,Amstetten,ARA Wohnhaus Melcher Kurt KG Ertl 2479 AM,3,1975,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,,,
2,1,Amstetten,ARA Wohnhaus Steiner KG Sonntagberg 2786 AM,4,1985,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,,,
3,1,Amstetten,ARA Raiffeisenlagerhaus KG Haag Stadt 2043 AM,8,1996,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,,,
4,1,Amstetten,ARA Bachler Herbert 2120 AM,8,1966,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4536,1,Wien Umgebung,ARA Withalm Hans 670 WU,20,1962,739782.0,345622.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4537,1,Wr. Neustadt-Land,ARA Eysank Lukas KG Hollenthon 4071 WB,8,2013,746822.0,268272.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4538,1,Zwettl,ARA Gretz KG Perndorf 4035 ZT,8,2003,658687.0,392714.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4539,1,Zwettl,ARA Böck KG Waldhausen 4908 ZT,10,2004,670487.0,376068.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
data.columns

Index([      'Unnamed: 0',           'BEZIRK',             'NAME',
         'EW60_BEWILLIGT', 'BEWILLIGUNGSJAHR',       'RECHTSWERT',
               'HOCHWERT',          'Mechan.',          'Biolog.',
               'Chemisch',           'Unbek.',          'Betrieb',
            'Stillgelegt',            'Unbek',                  1,
                        2,                  3,                  4,
                        5,                  6,                  7,
                        8,                  9,                 10,
                       11,                 12,                 13,
                       14,      'Unnamed: 28',      'Unnamed: 29',
            'Unnamed: 30',      'Unnamed: 31',      'Unnamed: 32',
            'Unnamed: 33',      'Unnamed: 34',      'Unnamed: 35',
            'Unnamed: 36',      'Unnamed: 37',      'Unnamed: 38',
            'Unnamed: 39',      'Unnamed: 40',      'Unnamed: 41',
            'Unnamed: 42',      'Unnamed: 43',      'Unnamed: 

In [76]:
data = cleaner(data, ["Unnamed: 0", "Unnamed: 28",
               "Unnamed: 29", "Unnamed: 30"])

#work on bautyp
bautyp = data[["Mechan.", "Biolog.", "Chemisch", "Unbek."]]
conditions_bau = [
    data["Unbek."] == 1,
    data["Mechan."] == 1,
    data["Biolog."] == 1,
    data["Chemisch"] == 1,
    (data["Biolog."] == 1) & (data["Mechan."] == 1)
]

outcomes_bau = [
    "Unbekannt", "Mech", "Bio", "Chem", "Bio"
]

data = logical_column(conditions_bau, outcomes_bau, data, 'bautyp')
data=col_dropper(data, bautyp)
data

#work on tech_type
col_list = ['3-k', 'Bel.', 'SBR', 'MBR', 'Tropf', 'RBC', 'Fest', 'Wirbel', 'BKF', 'PKA', 'Filtersack',
         'Kompost', 'Andere', 'Unbekannt']



change_colname_index(9, data,col_list)

,BEZIRK,NAME,EW60_BEWILLIGT,BEWILLIGUNGSJAHR,RECHTSWERT,HOCHWERT,Betrieb,Stillgelegt,Unbek,3-k,...,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,bautyp
0,Lilienfeld,ARA Wohnhaus Eberl KG Kerschenbach 1572 LF,4,1983,700050.0,323594.0,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,,,,,Unbekannt
1,Amstetten,ARA Wohnhaus Melcher Kurt KG Ertl 2479 AM,3,1975,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,,,,,Mech
2,Amstetten,ARA Wohnhaus Steiner KG Sonntagberg 2786 AM,4,1985,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,,,,,Bio
3,Amstetten,ARA Raiffeisenlagerhaus KG Haag Stadt 2043 AM,8,1996,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,,,,,Unbekannt
4,Amstetten,ARA Bachler Herbert 2120 AM,8,1966,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,,,,,Mech
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4536,Wien Umgebung,ARA Withalm Hans 670 WU,20,1962,739782.0,345622.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mech
4537,Wr. Neustadt-Land,ARA Eysank Lukas KG Hollenthon 4071 WB,8,2013,746822.0,268272.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bio
4538,Zwettl,ARA Gretz KG Perndorf 4035 ZT,8,2003,658687.0,392714.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bio
4539,Zwettl,ARA Böck KG Waldhausen 4908 ZT,10,2004,670487.0,376068.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bio


In [78]:
data.columns

Index([          'BEZIRK',             'NAME',   'EW60_BEWILLIGT',
       'BEWILLIGUNGSJAHR',       'RECHTSWERT',         'HOCHWERT',
                'Betrieb',      'Stillgelegt',            'Unbek',
                    '3-k',             'Bel.',              'SBR',
                    'MBR',            'Tropf',              'RBC',
                   'Fest',           'Wirbel',              'BKF',
                    'PKA',       'Filtersack',          'Kompost',
                 'Andere',        'Unbekannt',      'Unnamed: 31',
            'Unnamed: 32',      'Unnamed: 33',      'Unnamed: 34',
            'Unnamed: 35',      'Unnamed: 36',      'Unnamed: 37',
            'Unnamed: 38',      'Unnamed: 39',      'Unnamed: 40',
            'Unnamed: 41',      'Unnamed: 42',      'Unnamed: 43',
            'Unnamed: 44',      'Unnamed: 45',      'Unnamed: 46',
            'Unnamed: 47',      'Unnamed: 48',      'Unnamed: 49',
            'Unnamed: 50',      'Unnamed: 51',      'Unnamed: 

In [79]:
#same as later
data[data['Unbekannt']==1]['EW60_BEWILLIGT'].sum()

12763

# smae here using KKA sheet
